In [1]:
import keras
import numpy as np

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Pull Data

In [2]:
path = keras.utils.get_file(
    'nietzche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)

606208/600901 [==============================] - 1s 1us/step


In [4]:
text = open(path).read().lower()

In [6]:
print('number of characters: {}'.format(len(text)))

number of characters: 600893


## Prepare Data

In [10]:
maxlen = 60 # create sequences (sentences) of this length
step = 3 # create samples of `maxlen` sequences every `step` characters
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [20]:
print(sentences[0:5])

['preface\n\n\nsupposing that truth is a woman--what then? is the', 'face\n\n\nsupposing that truth is a woman--what then? is there ', 'e\n\n\nsupposing that truth is a woman--what then? is there not', '\nsupposing that truth is a woman--what then? is there not gr', 'pposing that truth is a woman--what then? is there not groun']


In [19]:
next_chars[0:5]

['r', 'n', ' ', 'o', 'd']

In [21]:
len(sentences)

200278

In [25]:
unique_chars = set(text)
chars = sorted(list(unique_chars))

In [26]:
print('{} unique characters'.format(len(chars)))

57 unique characters


In [29]:
char_indexes = dict((char, chars.index(char)) for char in chars) # create a map of indexes on the chars list

In [32]:
char_indexes['a']

27

In [33]:
chars[27]

'a'

### Vectorization

TODO: Use scikit or another preprossor

In [34]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

In [35]:
x.shape

(200278, 60, 57)

In [36]:
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [37]:
y.shape

(200278, 57)

In [39]:
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_indexes[char]] = 1
    y[i, char_indexes[next_chars[i]]] = 1

## Build Model

In [40]:
from keras import layers, models

In [41]:
model = models.Sequential()

In [42]:
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))

In [43]:
model.add(layers.Dense(len(chars), activation='softmax'))

In [44]:
model.compile(
    optimizer=keras.optimizers.RMSprop(lr=0.01),
    loss='categorical_crossentropy'
)

In [45]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [47]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indexes[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 134s 668us/step - loss: 1.6414
--- Generating with seed: "ian, half-german narrowness and stupidity in which
it has fi"
------ temperature: 0.2
ian, half-german narrowness and stupidity in which
it has find to the sense the be and the can be and the concerning to the fact to the fact the concerning the more to the such the consequentally the best for the man and soul and to the man and the concernal secument and the consequentally the precisely to the means to the an and the concerning the sense to the have to the any this the the can be and the percernal conceane to the for the conception of the 
------ temperature: 0.5
and the percernal conceane to the for the conception of the happere do to the excitenate conceane the the an are say moate to the make the result of fears it is an an and for the most consequention in this secomes also the sense of the consequently distanged in this even the an ancition. the to philosop

ifver for these sully," one of this slaste
defess,
it is a beare thrisher. enciejuably responsice,ty even titk of .bulfter texpressidely--if in their
gentaid with haveafren discividmed
charatrled in maldy is forated wiluloorance ligit of
her
beside bmolaldy, as ophility that, all (in infemanize gedien called of was sooo if not, a sulted
intert, w
epoch 5
Epoch 1/1
200278/200278 [==============================] - 134s 671us/step - loss: 1.4497
--- Generating with seed: "nies all success. "freedom of will"--that is the
expression "
------ temperature: 0.2
nies all success. "freedom of will"--that is the
expression of the soul, the profound and the sense of the soul. the present the soul. and and more the sense of the serve the sense of the more the experient of the same the sense of the soul. which the conscience of the sense of the development of the most and have the sense of the present and present the present the profound of the sense of the sense of the desires of the soul," and the

returne and alme? i much and event animan for "out
of the unterprationtly freedom, as desprearually the impulse of him
occasike: by which the uninholish subed
art they,
fore?""--to prosise
should force, formitul a
------ temperature: 1.2
subed
art they,
fore?""--to prosise
should force, formitul and delight)s (wes, ountically
moral or.

161. the actorce: am it not on soul
camtience the temmination--like comminer for him-et thereconexs--yout upon any
othercy vircoming in asharedn clases,"
bore the
question. backly greet
art which teicest orling alr
our europeanoting easely and
as an idisiparied!


edunence one, and through and deduce onlusion
tlhes on the ordemty, with dorm repleciated
phil
epoch 9
Epoch 1/1
200278/200278 [==============================] - 139s 696us/step - loss: 1.3997
--- Generating with seed: "cally honest, so
tedious! and was it ever otherwise? what th"
------ temperature: 0.2
cally honest, so
tedious! and was it ever otherwise? what the sense of the sense of the se

who are some soul and here and such every soul it we enthust the civilization and the germany and mankind of the states as a man and well discovers in the contempt the soul to say, and still and the futurian actions, when the got to the eyes of concerning the spirits,
where the experience
------ temperature: 1.0
 to the eyes of concerning the spirits,
where the experience
gate its overbued of the secicistary. the concerning the
belemple, the apepreful onei, the little under not "lander timms will--in first itseafed stases: may not become unvery may not weight throughout circumstress
of a whole foun interpostans when they mide comes
to arher than excets that a new to the other is philosophers and
we fre has from the nation, be gase mestically ten than itstles
trans
------ temperature: 1.2
s from the nation, be gase mestically ten than itstles
transhil soul! similism and spiritn of irrestumes with perversing does be indepred
to power, hence be
outbest immine itself revenge
dormantly
bay 

in themselves when they honoured the contrary of the sense of the other seeming in the relations of the world and the precisely and the whole sense of precisely the self-extrame the sense of the presence of the fact the sentiment in the precisely of the action of the sense of the sense of the fact that the presence of the same as and the good and there and the sense of the sense of the strength and the strength and the same and s
------ temperature: 0.5
he sense of the strength and the strength and the same and same and reverence and extrain at our highest the conscience of the action of the most such alm; of which one of the the connection of an act seeming in the presence of the presence of the fact that they are our spirit of desprive of the new former in the relations of herous of the last concepten, he always from the man of man, which is precisely the conceive of the precimene present, explanation t
------ temperature: 1.0
ecisely the conceive of the precimene present, explanatio

...

ion of the approceing and successman men, the history of the pigfioust time
thy state of a exalted succomed stricgl their existence, ealith only of this, every manners and intuition, by men refnesly
among it how but after on a strong a
manquility and prominations of the ha she will at the conscience
of end, as an atially roling
the late of ill experience. on the hangeroked with the
then without tongunable rate in errones the man who nameledly
sult afo, the
------ temperature: 1.2
ngunable rate in errones the man who nameledly
sult afo, the fact them rexigofarymeoum; neking too raced the
fatount are louted is glaitined.", i faith in its ownned
of higher likely of
the to the mans on the laws of his exception the spirit he les always with such the faith.--annbeding rate, which, in invention bualt righin, supposised,
man divions.--disclestion usually indepinks; it is upon the idorous
qui refugdity
from with them trafing cordest proposi
epoch 44
Epoch 1/1
200278/200278 [====================

se of the reason of the present present and sensual and such the whole expected and still present, to all the contemptive only of his detaul his problem of the viously of forment and the great superstituul of the perhaps allowed, they are conscience, the spirit of the worad of the most allowed intentional existent the present place of the problem of a or abemsens, and the boldless, in the expense of his own such an expecially that will truth. the both the 
------ temperature: 1.0
of his own such an expecially that will truth. the both the develop time yet ready if the age of great pricour, certain which the
fight treatow hear are of morality of its over which has presse" of error, expecianism,
still education of the burdening of "moralitiesn!" perhaps conulied by
all such more hadsures. this mude one of cross, among obere were the cheitudeals to puritible is
respect. atsome quite before the
such platosipoming,
or the degree when on
------ temperature: 1.2
e quite before the
such platos

handfuls of words, in which a more and sense of a spirit, the sense of the sense of the same that the sense of a demons to the same than the contempt and sense of the same that the same in the same is the sense of the sense of the soul is the sense of the sense of the same is the sense of the sense of the sense of a sense of the sense of a new conscience of the sense of the sense of the sense of the sense of the sense of the 
------ temperature: 0.5
 of the sense of the sense of the sense of the sense of the popular impisses of the contempt the a contempt enoting of the greatest in the danger and sense of the suptration of the other will and not feel were to the most talk and sorts of a god and most enough and sense of the same than the sense of the spirit in the discoverous of the struce, the destiny to the second to a sphen it is a sort of accountary of itself or a death to be a more and metaphysica
------ temperature: 1.0
accountary of itself or a death to be a more and metaphysical

io usilier--this has and preciolind, a has valoroisikes, its beleted when believe thrumative
holy exampation allogsing toke, ran",--but mode whe
epoch 55
Epoch 1/1
200278/200278 [==============================] - 162s 807us/step - loss: 1.4470
--- Generating with seed: "e mightiest men have hitherto always bowed reverently before"
------ temperature: 0.2
e mightiest men have hitherto always bowed reverently beforeevy the consequence of the sense of the same the same that the same the object the sense of the sense of the spiritual respect the any other interpretation of the sense of the sense of the ascently to the sense of the sense of the sense of the same and such a suffered that the sense of the sense of the sense of the sense of the sense of the same that is the man in the same the sense of the other e
------ temperature: 0.5
he same that is the man in the same the sense of the other envelus in the develod of the sickly for the indifferent fact they also the sense of the objecually

is modes amongs other not--isilitation: i i authors
part ofe
onmy is certain understand trde
from pherok it is withing its la-criscant inhertouage, oen thems middly can
repulatisi curtovieation, granted.=--this onnerly whom that for moraliciation. i gaining would balt for,  gives so repreeminity to a particularil responcious to there
all of philosophical greatles attack
one's interlansh of precisely moraled shaules all
epoch 59
Epoch 1/1
200278/200278 [==============================] - 148s 737us/step - loss: 1.3381
--- Generating with seed: "h
contradiction. according to its explanation, there is, str"
------ temperature: 0.2
h
contradiction. according to its explanation, there is, strange and the most spirit the moral of the work of the free spirit of the present with the present will of the most spirit and the conception of the present the conception of the world to the man with the present such a specialist and man is not the moral of the fact that is the the word of the moral of t